In [2]:
import unittest
from pstats import Stats
import cProfile
import numpy as np
from functools import partial
from cost_functions import V_a, gradV_a, V_b, gradV_b
from numpy.linalg import norm, eig

In [74]:
def _step_size(p, x, s, gamma=1.5, mu=0.8):
    """Armijo algorithm for computing step size
    Parameters
    ----------
    gamma : float
        Parameter for increasing step size
    mu : float
        Parameter for decreasing step size
    Returns
    -------
    float
        Step size
    """

    w = 1  # Default step size

    k_g = 0  # Power of gamma
    k_m = 0  # Power of mu

    # Precompute cost and gradient to save time
    vx = p.cost(x)
    gx_s = p.grad(x) @ s

    def v_bar(w):
        return vx + 0.5 * w * gx_s

    while p.cost(x + gamma**k_g * s) < v_bar(gamma**k_g):
        k_g += 1
        w = gamma**k_g

    while p.cost(x + mu**k_m * gamma**k_g * s) > v_bar(mu**k_m * gamma**k_g):
        k_m += 1
        w = mu**k_m * gamma**k_g

    return w

def steepest_descent(p, x, tol=1e-6, max_iter=999, hist=False):
    """Steepest descent optimization algorithm
    Parameters
    ----------
    p : Problem
        Problem to minimize
    x : 2D numpy column array of floats
        Initial guess at minimum
    tol : float
        When norm of gradient goes below this value, iteration stops
    max_iter : int
        Absolute maximum number of iterations before giving up and returning x
    hist : bool
        If True, returns array with value of x at every iteration. If False,
        just returns last x value.o
    Returns
    -------
    2D or 3D numpy column array of floats
        If hist is False, returns 2D numpy colmn array containing minimizing
        x of problem. Otherwise returns a 3D numpy array containing every
        value of x along the way.
    """

    i = 0
    x_hist = []
    while np.linalg.norm(p.grad(x)) > tol:
        if i > max_iter:
            break
        s = -p.grad(x).T
        w = _step_size(p, x, s)
        x_hist.append(x)
        x = x + w * s
        i += 1

    return x if not hist else np.array(x_hist)

def _fd_grad(f, x, h=1e-8):
    """Finite difference approximation of the gradient
    Parameters
    ----------
    f : function of x (2D numpy column array)
        Function whose gradient you want to evaluate
    x : 2D numpy column array
        Point where you want to evaluate the gradient
    h : float
        Step size
    Returns
    -------
    2D numpy row array
        Gradient (which is a row vector)
    """

    dim = np.max(np.shape(x))
    grad_gen = ((f(x + h * np.eye(dim)[:, [i]]) - f(x)) / h
               for i in range(0, dim))
    grad = np.expand_dims(np.fromiter(grad_gen, np.float64), axis=0)
    return grad


class Problem:
    """Optimization problem"""
    def __init__(self, cost, grad=None, grad_step=None,
                 eq_const=None, ineq_const=None):
        """Constructor for Problem object
        Creates a Problem object for use with optimization algorithms.
        Parameters
        ----------
        cost : function whose input is 2D numpy column array
            Objective function of problem
        grad : function whose input is 2D numpy column array
            Gradient function of problem. If not specified, finite difference
            is used
        grad_step : float
            Step size for finite difference gradient. If not specified, (and
            no analytic gradient is given) 1e-8 is used
        eq_const : list of functions
            List of functions that form equality constraints for problem
        ineq_const : list of functions
            List of functions that form inequality constraints for problem
        """
        self._cost = cost
        # Check presence of gradient function
        if grad is None and grad_step is None:
            # No gradient function, default step size
            self._grad_step = 1e-8
            self._grad = partial(_fd_grad, self.cost, h=self._grad_step)
        elif grad is None and grad_step is not None:
            # No gradient function, specified step size
            self._grad_step = grad_step
            self._grad = partial(_fd_grad, self.cost, h=self._grad_step)
        elif grad is not None:
            # Gradient given, no need for step size
            self._grad_step = None
            self._grad = grad
        # Check presence of constraints
        if eq_const is not None:
            self._eq_const = eq_const
        else:
            self._eq_const = None
        if ineq_const is not None:
            self._ineq_const = ineq_const
        else:
            self._ineq_const = None

    def cost(self, x=None):
        """Cost of Problem
        If x is given, returns cost at x. Otherwise returns cost function.
        Parameters
        ----------
        x : 2D numpy column array
            Point at which to evaluate cost
        Returns
        -------
        float or function
            If x was given, returns cost at x. Otherwise returns cost function.
        """
        if x is not None:
            return self._cost(x)
        else:
            return self._cost

    def grad(self, x=None):
        """Gradient of Problem
        If x is given, returns gradient at x. Otherwise returns gradient
        function.
        Parameters
        ----------
        x : 2D numpy column array
            Point at which to evaluate gradient
        Returns
        -------
        2D numpy row array or function
            If x was given, returns gradient at x. Otherwise returns gradient
            function.
        """
        if x is not None:
            return self._grad(x)
        else:
            return self._grad

    def eq_const(self, x=None):
        """ Equality constraints of Problem
        If x is given, returns column array of constraints evaluated
        at x. Otherwise returns column array of functions.
        Parameters
        ----------
        x : 2D numpy column array
            Point at which to evaluate constraints
        Returns
        -------
        2D numpy column array of floats or functions
            If x was given, returns column array of costs at x. Otherwise
            returns column array of functions
        """
        if self._eq_const is not None:
            if x is not None:
                return np.array([[eq(x)] for eq in self._eq_const])
            else:
                return np.array([eq for eq in self._eq_const])
        else:
            return None

    def ineq_const(self, x=None):
        """ Inequality constraints of Problem
        If x is given, returns column array of constraints evaluated
        at x. Otherwise returns column array of functions.
        Parameters
        ----------
        x : 2D numpy column array
            Point at which to evaluate constraints
        Returns
        -------
        2D numpy column array of floats or functions
            If x was given, returns column array of costs at x. Otherwise
            returns column array of functions
        """
        if self._ineq_const is not None:
            if x is not None:
                return np.array([[ineq(x)] for ineq in self._ineq_const])
            else:
                return np.array([ineq for ineq in self._ineq_const])
        else:
            return None

    def num_eq_const(self):
        """Returns number of equality constraints"""
        if self._eq_const is not None:
            return np.max(np.shape(self._eq_const))
        else:
            return 0

    def num_ineq_const(self):
        """Returns number of inequality constraints"""
        if self._ineq_const is not None:
            return np.max(np.shape(self._ineq_const))
        else:
            return 0


In [85]:
def augmented_lagrange(p, x0, tol=1e-6, tol_const=1e-6, sigma_max=1e12, hist=False):

    def phi(p, lmb, sgm, x):
        """Unconstrained problem"""
        cost = p.cost(x)

        n_e = p.num_eq_const()
        n_i = p.num_ineq_const()
        n_c = n_e + n_i

        lmb_e = lmb[0:n_e, :]
        lmb_i = lmb[n_e:n_c, :]
        sgm_e = sgm[0:n_e, :]
        sgm_i = sgm[n_e:n_c, :]


        if p.eq_const() is not None:
            c_e = p.eq_const(x)
            cost = cost - sum(lmb_e * c_e) + 0.5 * sum(sgm_e * c_e**2)
        if p.ineq_const() is not None:
            c_i = p.ineq_const(x)
            p_i = np.array([-lmb_i[i] * c_i[i] + 0.5 * sgm_i[i] * c_i[i]**2 
                            if c_i[i] <= lmb_i[i] / sgm_i[i] 
                            else -0.5 * lmb_i[i]**2 / sgm_i[i] 
                            for i in range(0, n_i)])
            cost = cost + sum(p_i)
        return cost

    x_hist = []

    n_e = p.num_eq_const()
    n_i = p.num_ineq_const()
    n_c = n_e + n_i

    lmb = np.zeros((n_c, 1))
    sgm = np.ones((n_c, 1))

    x = x0
    c = 1e12 * np.ones((n_c, 1))

    while np.linalg.norm(c) > tol_const:
        # Create new problem to solve, but unconstrained
        
        up = Problem(partial(phi, p, lmb, sgm))

        print(phi(up, lmb, sgm, x))


        x_hist.append(x)
        x = steepest_descent(up, x0, tol=tol)


        # Concatenate costs
        c_prv = c
        c_e = p.eq_const(x)
        c_i = p.ineq_const(x)
        if c_e is not None and c_i is not None:
            c = np.concatenate((c_e, c_i), axis=0)
        elif c_e is not None:
            c = c_e
        elif c_i is not None:
            c = c_i

        # Make sure sigma is not too big
        if any(sgm >= sigma_max):
            break

        # Update sigma
        if np.linalg.norm(c, np.inf) > 0.25 * np.linalg.norm(c_prv, np.inf):
            for i in range(0, n_c):
                if np.abs(c[i]) > 0.25 * np.linalg.norm(c_prv, np.inf):
                    sgm[i] *= 10
            continue

        lmb = lmb - (sgm * c)
    print(x)
    return x if not hist else np.array(x_hist)

In [86]:

def setUp():
    v = lambda x: -x[0, 0] * x[1, 0]
    h1 = lambda x: -x[0, 0] - x[1, 0]**2 + 1
    h2 = lambda x: x[0, 0] + x[1, 0]
    return Problem(v, ineq_const=[h1, h2])

def test_aug_lag(p):
    x0 = np.array([[10], [1]])
    x = augmented_lagrange(p, x0, tol=1e-4, tol_const=1e-4)
    return x


In [87]:
x = test_aug_lag(setUp())
x

[40.]
[-1.91416744]
[-0.60024072]
[-0.28523623]
[-0.38865798]
[-0.38695383]
[-0.38678342]
[-0.38676638]
[-0.38676467]
[-0.3867645]
[-0.38676449]
[-0.38676449]
[-0.38676449]
[-0.38676449]
[[0.65361728]
 [0.57165379]]


array([[0.65361728],
       [0.57165379]])

In [65]:
print(x)

None
